# 実験の再現性を検証する

ワークフローの再現性に関わる必要なメタデータをパッケージング（整理）し、Sapporoを用いて実験の再現性を検証します。
検証内容は以下の3点です。  
- RO-Crate 内の値に基づいて、ワークフローが再実行可能であること
- 再実行のステータスが、RO-Crate に記載されているステータスと同一であること
- 再実行で得られた結果ファイルのサイズ・チェックサムが、RO-Crate の値と同一であること

### パッケージング対象のメタデータについて
パッケージング対象のメタデータ項目は以下です。  

|項目|説明|
|:-:|:-:|
|SapporoRun|Sapporoでワークフローを再実行し、既存のワークフロー実行結果と比較するために必要なメタデータ|
|Dataset|既存のワークフロー実行結果データセットのメタデータ|
|File|既存のワークフロー実行結果ファイルのメタデータ|
  
再現性に関わるSapporoRun項目として以下の要素を整理します。

|要素|必要性|説明|
|:-:|:-:|:-:|
|workflow_params|任意|ワークフロー実行のためのパラメータ|
|workflow_type|任意|CWLやNextflowなどといったワークフロー言語の種別|
|workflow_type_version|任意|ワークフロー言語のバージョン|
|tags|任意|workflow_params以外の任意のメタデータをキー・バリューのマップにしたもの|
|workflow_engine_name|必須|ワークフローエンジン名|
|workflow_engine_parameters|任意|追加のパラメータをワークフローエンジンに送信できる|
|workflow_url|任意|CWLまたはWDLのドキュメントのURL|
|workflow_name|任意|Sapporoに登録されたワークフロー実行時の名前|
|workflow_attachment|任意|ワークフローの実行に必要な入力データをSapporoにアップロードするためのファイル名とファイルのURL|
|sapporo_location|必須|利用するSapporoサービスのURL|
|state|必須|再実行対象の実行済みワークフローのステータス|
|outputs|必須|再実行対象の実行済みワークフローの出力結果データセット|

### 検証結果について
検証結果は必ず同期する必要はありません。この環境で検証結果を確認いただいたあとに、GIN-forkへ同期するか破棄するか選択いただけます。  
検証結果を同期する場合は、研究リポジトリの直下の**validation_results**フォルダに格納されます。タスク実行後に検証結果を確認する際はvalidation_resultsフォルダ配下の**results.json**をご確認ください。  
同期しない場合は、検証結果は破棄されます。

研究フロートップページに戻る場合は[こちら](../../base_FLOW.ipynb)。新規タブで開きます。  

## 0. 研究リポジトリ名を確認する

以下のセルを実行すると、この実験実行環境で操作する研究リポジトリ名を確認できます。 

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
utils.show_name('green')

## 1. 研究データを最新化する

In [ ]:
!datalad update -s gin --merge

## 2. 再現性を確認したい実験パッケージを選択する

以下のセルを実行して、再現性を検証したい実験パッケージを選んでください。  
選択を間違えた場合は、このセルを再実行してください。

In [ ]:
import json
from ipywidgets import Dropdown, Button
from IPython.display import clear_output

# 実験パッケージ名を取得
with open('/home/jovyan/pipeline.json', 'r') as f:
    pipeline = json.load(f)

# 再現する実験パッケージを選択
def on_click_callback(clicked_button: Button) -> None:
    global package
    package=dropdown.value
    clear_output()
    print("入力を受けつけました：", package)

dropdown = Dropdown(
    options=pipeline,
    description='実験パッケージ名:',
    disabled=False,
)

button = Button(description='入力完了')
button.on_click(on_click_callback)
display(dropdown, button)

## 3. 実験パッケージのデータを取得する

In [ ]:
import os

package_dir = "/home/jovyan/experiments/" + package
result_dir = package_dir + "/output_data"

os.chdir(package_dir)
!datalad get .

## 4. 再現性に関わるメタデータを検証する

In [ ]:
import os
import json
import subprocess
import logging

!mkdir -p '/home/jovyan/.tmp/demo'
os.chdir('/home/jovyan/.tmp/demo')
cmd = ["python3", "/home/jovyan/nii-dg/sapporo_example/package_ro_crate.py", "http://dg02.dg.rcos.nii.ac.jp:3000", result_dir]
try:
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode != 0:
        raise Exception(result)
except Exception as e:
    logging.error('メタデータのパッケージングに失敗しました。')
    raise Exception(str(e))
else:
    print("メタデータをパッケージングしました。")
    !cat ro-crate-metadata.json


In [ ]:
from urllib import parse
import requests
import os
import json
import logging

with open("/home/jovyan/.tmp/demo/ro-crate-metadata.json") as f:
    ro_crate = json.load(f)

try:
    # generate url_for_request_validation
    url_for_request_validation = parse.urlunparse((
        'http',
        'dg02.dg.rcos.nii.ac.jp:443', 
        'validate',
        '',
        '',
        ''
    ))
    # request validation
    headers = {'content-type': 'application/json'}
    response = requests.post(url_for_request_validation, data=json.dumps(ro_crate), headers=headers)
    result = response.json()
except Exception as e:
    logging.error('検証サービスに接続できません。')
    raise Exception(str(e))
else:
    if response.status_code == requests.codes.ok:
        os.chdir(os.environ['HOME'])
        from WORKFLOWS.utils.tmp_validation import tmp_validation
        request_id = result['request_id']
        tmp_validation.save_request_id(request_id)
        print('正常に検証が依頼されました。次の処理にお進みください。\nリクエストID：' + request_id)
    elif response.ok == False:
        logging.error('異常が発生しました。担当者にお問い合わせください。')
        raise Exception(result['message'])
        

## 5. 検証結果を確認する

In [ ]:
from WORKFLOWS.utils.tmp_validation import tmp_validation

tmp_validation.get_validation_results()

## 6. 研究リポジトリに同期する

このタスクの実行結果を研究リポジトリに同期します。  
検証結果を同期するか破棄するかは、「5.1. 検証結果を研究リポジトリに同期するか選択する」で選択できます。

### 6.1. 検証結果を研究リポジトリに同期するか破棄するかを選択する

In [ ]:
import panel as pn

pn.extension()
column = pn.Column()

def save_selection_result(event):
    global need_sync
    done_button.button_type = "success"
    done_button.name = "選択完了しました。次の処理にお進みください。"
    need_sync = True if select.value == 1 else False

select = pn.widgets.Select(name='検証結果を同期するか破棄するかを選択した後、完了ボタンをクリックしてください。', options={'同期する': 1, '同期せずに破棄する':2})
done_button = pn.widgets.Button(name= "選択を完了する", button_type= "primary")
done_button.on_click(save_selection_result)
column.append(select)
column.append(done_button)
column

### 6.2. 研究リポジトリに同期する

研究リポジトリにこのタスクの実行結果を同期します。  
「5.1. 検証結果を研究リポジトリに同期するか破棄するかを選択する」で同期せずに破棄するを選択した場合は、検証結果は同期されずこのファイルの実行結果のみが同期されます。

In [ ]:
import os
from IPython.display import display, Javascript
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
os.chdir(os.environ['HOME'])
from WORKFLOWS.utils.tmp_validation import tmp_validation

tmp_validation.operate_validation_results(need_sync)
git_path = ['/home/jovyan/WORKFLOWS/FLOW/03_after_experiments_phase/base_validate_repeatability.ipynb']
if need_sync == True:
    validation_result_path = tmp_validation.fetch_validation_result_path()
    git_path.append(validation_result_path)

display(Javascript('IPython.notebook.save_checkpoint();'))
is_ok = utils.syncs_with_repo(git_path=git_path, gitannex_path=None, gitannex_files=None, message='実験パッケージ' + package + 'の再現性検証')
if is_ok:
    tmp_validation.delete_verification_results_and_request_id()

## 7. 研究フロートップページに遷移する

以下のセルを実行し、表示されるリンクをクリックして研究フロートップページに戻ってください。

In [ ]:
from IPython.display import display, HTML, Javascript
display(HTML("<a href='../../base_FLOW.ipynb'>研究フロートップページに遷移する</a>"))
display(Javascript('IPython.notebook.save_checkpoint();'))